In [72]:
import json
import time
import re
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [73]:
# Function to clean and tokenize titles
def tokenize_title(title):
    return set(re.sub(r"[^\w\s]", "", title.lower()).split())

# Function to check word match percentage
def is_match(input_title, paper_title, threshold=10):
    input_tokens = tokenize_title(input_title)
    paper_tokens = tokenize_title(paper_title)
    
    common_words = input_tokens.intersection(paper_tokens)
    match_percentage = (len(common_words) / len(input_tokens)) * 100 if input_tokens else 0
    
    return match_percentage >= threshold, match_percentage


In [74]:
def supervisor_agent(search_query):
    print(f"\n🔶 Supervisor Agent: Managing Research on '{search_query}'...")
    
    papers = generation_agent(search_query)
    feasible_papers = reflection_agent(papers, search_query)
    ranked_papers = ranking_agent(feasible_papers)
    improved_papers = evolution_agent(ranked_papers)
    final_paper = proximity_agent(improved_papers)
    
    meta_review_agent()
    
    print("\n✅ Final Optimized Paper:", final_paper)
    return final_paper


In [75]:
import json
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

def generation_agent(search_query):
    print(f"\n🔹 Generation Agent: Fetching Papers for '{search_query}'...")

    # Setup Selenium with random user-agent to avoid bot detection
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"
    ]

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")
    options.add_argument("--log-level=3")  
    options.add_argument(f"user-agent={random.choice(user_agents)}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # List of academic websites
    websites = {
        "Google Scholar": {
            "url": f"https://scholar.google.com/scholar?q={search_query.replace(' ', '+')}",
            "title_xpath": "//h3[@class='gs_rt']/a",
            "link_xpath": "//h3[@class='gs_rt']/a",
        },
        "CORE (Research Papers)": {
            "url": f"https://core.ac.uk/search?q={search_query.replace(' ', '+')}",
            "title_xpath": "//div[contains(@class, 'search-result')]//h3/a",
            "link_xpath": "//div[contains(@class, 'search-result')]//h3/a",
        }
    }

    data_list = []

    for site, details in websites.items():
        print(f"🔍 Searching: {site}...")

        driver.get(details["url"])
        time.sleep(10)  # Increased wait time for JavaScript-heavy pages

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, details["title_xpath"]))
            )
        except TimeoutException:
            print(f"⚠️ Timeout on {site}, skipping...")
            continue

        # ✅ Fetch the number of papers dynamically
        total_papers = min(len(driver.find_elements(By.XPATH, details["title_xpath"])),
                           len(driver.find_elements(By.XPATH, details["link_xpath"])))

        if total_papers == 0:
            print(f"⚠️ No results found for {site}.")
            continue

        print(f"✅ Extracted {total_papers} papers from {site}.")

        for i in range(total_papers):
            try:
                # ✅ Fix: Fetch fresh elements dynamically in each loop iteration
                title_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, details["title_xpath"]))
                )
                link_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, details["link_xpath"]))
                )

                title_text = title_element.text.strip() if title_element else "Title not found"
                link = link_element.get_attribute("href") if link_element else "No link available"

                # Fix CORE relative links
                if site == "CORE (Research Papers)" and link.startswith("/display/"):
                    link = "https://core.ac.uk" + link

                abstract_text = "Abstract not found"

                # ✅ Visit each paper's link to fetch full abstract
                if link != "No link available":
                    try:
                        driver.get(link)
                        time.sleep(5)  # Wait for page load

                        # ✅ Locate and extract abstract (Different websites have different structures)
                        if "core.ac.uk" in link:
                            abstract_xpath = "//div[contains(@class, 'abstract')]"
                        elif "ncbi.nlm.nih.gov" in link:
                            abstract_xpath = "//div[@class='abstr']//p"
                        elif "journals.aps.org" in link:  # Example for APS journal
                            abstract_xpath = "//div[@class='abstract']/p"
                        else:
                            abstract_xpath = "//p[contains(text(), 'Abstract') or contains(text(), 'abstract')]"

                        try:
                            abstract_element = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, abstract_xpath))
                            )
                            abstract_text = abstract_element.text.strip()
                        except (NoSuchElementException, TimeoutException):
                            print(f"⚠️ No abstract found on {link}, skipping...")

                    except Exception as e:
                        print(f"⚠️ Error loading page {link}: {e}")

                # ✅ Skip papers without abstracts
                if abstract_text == "Abstract not found":
                    print(f"⚠️ Skipping {title_text} (No abstract)")
                    continue  # Skip this paper

                # ✅ Store the data in JSON format
                data_list.append({
                    "site": site,
                    "title": title_text,
                    "link": link,
                    "abstract": abstract_text
                })

            except StaleElementReferenceException:
                print(f"⚠️ Stale element detected, refetching elements...")
                continue  # Skip this iteration and retry with fresh elements
    
    driver.quit()

    # ✅ Save data to a JSON file
    json_filename = "scraped_papers.json"
    with open(json_filename, "w", encoding="utf-8") as json_file:
        json.dump(data_list, json_file, indent=4, ensure_ascii=False)

    print(f"\n📁 Data saved successfully in {json_filename}!")
    return data_list


In [76]:
def reflection_agent(papers, search_query):
    print("\n🔹 Reflection Agent: Filtering Relevant Papers...")
    
    filtered_papers = [paper for paper in papers if is_match(search_query, paper["title"])[0]]
    
    print("✅ Feasible Papers:", len(filtered_papers))
    return filtered_papers


In [77]:
def ranking_agent(papers):
    print("\n🔹 Ranking Agent: Scoring Papers...")
    
    ranked_papers = sorted(papers, key=lambda x: random.randint(1, 100), reverse=True)
    
    print("🏆 Top Ranked Paper:", ranked_papers[0] if ranked_papers else "No papers found")
    return ranked_papers


In [78]:
def evolution_agent(papers):
    print("\n🔹 Evolution Agent: Enhancing Selection...")
    
    improved_papers = [paper for paper in papers if "Review" in paper["title"] or "Survey" in paper["title"]]
    
    print("🚀 Improved Selection:", improved_papers)
    return improved_papers if improved_papers else papers


In [79]:
def proximity_agent(papers):
    print("\n🔹 Proximity Agent: Checking Past Research Similarity...")
    
    past_research = ["AI Chatbots", "Self-Driving Cars", "Smart Homes"]
    best_paper = papers[0] if papers else None

    if best_paper and any(topic in best_paper["title"] for topic in past_research):
        print("⚠️ Paper is similar to past research!")
    
    return best_paper


In [80]:
def meta_review_agent():
    print("\n🔹 Meta-Review Agent: Final Performance Check...")
    
    print("📊 Papers Processed: ✅ Completed!")


In [81]:
search_query = input("Enter your research topic: ").strip()
final_paper = supervisor_agent(search_query)

print("\n🎯 Best Retrieved Research Paper:", final_paper)



🔶 Supervisor Agent: Managing Research on 'quantum learning'...

🔹 Generation Agent: Fetching Papers for 'quantum learning'...
🔍 Searching: Google Scholar...
✅ Extracted 10 papers from Google Scholar.
⚠️ No abstract found on https://www.researchgate.net/profile/Ron-Chrisley/publication/305848278_Quantum_Learning/links/57a3736408aefe6167a5cf5c/Quantum-Learning.pdf, skipping...
⚠️ Skipping Quantum learning (No abstract)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x005AC7F3+24435]
	(No symbol) [0x00532074]
	(No symbol) [0x004006E3]
	(No symbol) [0x00448B39]
	(No symbol) [0x00448E8B]
	(No symbol) [0x00491AC2]
	(No symbol) [0x0046D804]
	(No symbol) [0x0048F20A]
	(No symbol) [0x0046D5B6]
	(No symbol) [0x0043C54F]
	(No symbol) [0x0043D894]
	GetHandleVerifier [0x008B70A3+3213347]
	GetHandleVerifier [0x008CB0C9+3295305]
	GetHandleVerifier [0x008C558C+3271948]
	GetHandleVerifier [0x00647360+658144]
	(No symbol) [0x0053B27D]
	(No symbol) [0x00538208]
	(No symbol) [0x005383A9]
	(No symbol) [0x0052AAC0]
	BaseThreadInitThunk [0x75875D49+25]
	RtlInitializeExceptionChain [0x7798CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x7798CDC1+561]
